### Visualizing a 2D simulation model
To visualize a 2D matrix changing over time we can use a simple 2D ["stop motion" animation](https://en.wikipedia.org/wiki/Animation).
Luckily, `matplotlib` makes animating a 2D plot fairly straight-forward.

Here is a generic class that animates any 2D model that provides step() and draw() methods, and uses matplotlib to render its state.
It provides methods to `animate()` the model directly in the output display or `show()` the animation in an interactive viewer.

See [Matplotlib animation in Jupyter](https://www.allendowney.com/blog/2019/07/25/matplotlib-animation-in-jupyter/) by our textbook author for a discussion of the pros and cons of various animation techniques in jupyter notebooks.

In [1]:
%matplotlib inline

import time
from typing import Protocol

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import animation
import numpy as np

# Configure matplotlib's animation library to work in the browser.
matplotlib.rc('animation', html='jshtml')

In [2]:
class Simulation(Protocol):
    """ A Simulation is any object that provides methods to step() and draw() itself. """
    def step(self) -> None:
        """ Step the simulation model forward """
        ...
    def draw(self, axes, step:str) -> None:
        """ Draw the current model state on the given axes.  Current simulation step is provided for label """

In [3]:
class Animation2D:
    """
      Animates any 2D model with a step() method and a draw() method, using matplotlib
      model.step() should take no parameters - just step the model forward one step.
      model.draw() should take 2 parameters, the matpltolib axes to draw on and an integer step number

      See https://www.allendowney.com/blog/2019/07/25/matplotlib-animation-in-jupyter/
          for a discussion of the pros and cons of various animation techniques in jupyter notebooks
    """

    def __init__(self, model:Simulation, frames=50, steps_per_frame=1, figsize=(8, 8)):
        """
        :param model: the simulation object to animate, with step() and draw(axes, step) methods
        :param frames: number of animation frames to generate
        """
        self.model = model
        self.frames = frames
        self.steps_per_frame = steps_per_frame
        self.fig, self.ax = plt.subplots(figsize=figsize)

    def animation_step(self, step):
        """ Step the model forward and draw the plot """
        if step > 0:
            for _ in range(self.steps_per_frame):
                self.model.step()
        self.model.draw(self.ax, step=step * self.steps_per_frame)

    def show(self):
        """ return the matplotlib animation object, ready for display """
        anim = animation.FuncAnimation(self.fig, self.animation_step, frames=self.frames)
        plt.close()  # this ensures the last frame is not shown as a separate plot
        return anim

    def animate(self, interval=None):
        """ Animate the model simulation directly in the notebook display block """
        from IPython.display import clear_output
        try:
            for i in range(self.frames):
                clear_output(wait=True)  # clear the IPython display
                self.ax.clear()  # clear old image from the axes (fixes a performance issue)
                plt.figure(self.fig)  # add the figure back to pyplot ** sigh **
                self.animation_step(i)
                plt.show()  # show the current animation frame (pyplot then closes and throws away figure ** sigh **)
                if interval:
                    time.sleep(interval)
        except KeyboardInterrupt:
            pass

In [4]:
# Copy-pasted from 01-data-visualization notebook.

def plot_2d_array(array, axes=None, title='', cmap='viridis', **options):
    """ 
    Plot the 2D array as an image on the given axes  1's will be dark blue, 0's will be light blue. 

    :param axes: the axes to plot on, or None to use the `plt.gca()` (current axes)
    :param options: keyword arguments passed directly to `plt.imshow()`
           see https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.imshow.html
    """
    axes = axes or plt.gca()  # If not axes are provided, draw on current axes
    axes.set_title(title)
    # Turn off axes labels and tick marks
    axes.tick_params(axis='both', which='both', bottom=False, top=False, left=False, right=False,
                     labelbottom=False, labeltop=False, labelleft=False, labelright=False, )
    # Defaults for displaying a "matrix" with hard-pixel boundaries and (0,0) at top-left
    options = {**dict(interpolation='nearest', origin='upper'), **options}
    axes.imshow(array, cmap=cmap, **options)


In [5]:
# Sample "best practices" for producing an animation of a simulation model

class RandomModel:
    """ An 2D matrix where each cell grows and or dies back at random """

    def __init__(self, n_rows, n_cols=None):
        """ Construct a n_rows x n_cols 2D CA """
        n_cols = n_cols or n_rows  # i.e., n_cols if n_cols is not None and n_cols != 0 else n_rows
        self.state = np.random.randint(-5, 5, size=(n_rows, n_cols), dtype=np.int8)

    def step(self):
        """Executes one time step. """
        growth = np.random.randint(-2, 2, size=self.state.shape, dtype=self.state.dtype)
        self.state = self.state + growth

    def draw(self, axes=None, step=''):
        axes = axes or plt.gca()
        plot_2d_array(self.state, axes, f"Random fluctuations at step {step}")
        
anim = Animation2D(model=RandomModel(100))
anim.show()